In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import datetime

In [2]:
NORS_df = pd.read_csv('../data/NORS.csv')

C:\Users\rayna\AppData\Local\Temp\ipykernel_4208\874844664.py:1: DtypeWarning: Columns (8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  NORS_df = pd.read_csv('../data/NORS.csv')


In [3]:
NORS_df.head()

,Year,Month,State,Primary Mode,Etiology,Serotype or Genotype,Etiology Status,Setting,Illnesses,Hospitalizations,Info On Hospitalizations,Deaths,Info On Deaths,Food Vehicle,Food Contaminated Ingredient,IFSAC Category,Water Exposure,Water Type,Animal Type
0,1971,2,California,Water,Copper,NaN,Confirmed,Restaurant,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
1,1971,6,Arkansas,Water,Hepatitis A,NaN,Confirmed,Store,98,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Other,NaN
2,1971,6,Missouri,Water,Unknown,NaN,Suspected,Subdivision/Neighborhood,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
3,1971,6,Alabama,Water,Selenium,NaN,Confirmed,Unknown,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Individual/Private,NaN
4,1971,6,Vermont,Water,Unknown,NaN,Suspected,Community/municipality,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN


In [4]:
NORS_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66713 entries, 0 to 66712
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Year                          66713 non-null  int64  
 1   Month                         66713 non-null  int64  
 2   State                         66713 non-null  object 
 3   Primary Mode                  66713 non-null  object 
 4   Etiology                      50375 non-null  object 
 5   Serotype or Genotype          16470 non-null  object 
 6   Etiology Status               50375 non-null  object 
 7   Setting                       60804 non-null  object 
 8   Illnesses                     66713 non-null  object 
 9   Hospitalizations              58155 non-null  float64
 10  Info On Hospitalizations      58480 non-null  object 
 11  Deaths                        58785 non-null  float64
 12  Info On Deaths                58463 non-null  object 
 13  F

In [5]:
NORS_df.duplicated().sum()

np.int64(1421)

In [6]:
NORS_df['Primary Mode'].unique()

array(['Water', 'Food', 'Person-to-person', 'Indeterminate/unknown',
       'Animal contact',
       'Environmental contamination other than food/water'], dtype=object)

In [7]:
duplicated_rows = NORS_df[NORS_df.duplicated()]
print(duplicated_rows)

       Year  Month           State           Primary Mode           Etiology  \
521    1983      7       Minnesota                  Water  Norovirus unknown   
1037   1998      1            Ohio                   Food                NaN   
1049   1998      1      Washington                   Food                NaN   
1054   1998      1            Ohio                   Food                NaN   
1101   1998      1         Florida                   Food                NaN   
...     ...    ...             ...                    ...                ...   
66395  2023     11  South Carolina       Person-to-person  Norovirus unknown   
66473  2023     12       Wisconsin       Person-to-person  Norovirus unknown   
66532  2023     12            Iowa  Indeterminate/unknown                NaN   
66660  2023     12       Minnesota       Person-to-person  Norovirus unknown   
66661  2023     12       Minnesota       Person-to-person  Norovirus unknown   

      Serotype or Genotype Etiology Sta

In [8]:
NORS_df.drop_duplicates(keep='first', inplace=True)
NORS_df.duplicated().sum()

np.int64(0)

In [9]:
NORS_df['Month'] = NORS_df['Month'].astype('str').replace({'1': '01', '2': '02', '3': '03', '4': '04', '5': '05', '6': '06', '7': '07', '8': '08', '9': '09'})
NORS_df.head()

,Year,Month,State,Primary Mode,Etiology,Serotype or Genotype,Etiology Status,Setting,Illnesses,Hospitalizations,Info On Hospitalizations,Deaths,Info On Deaths,Food Vehicle,Food Contaminated Ingredient,IFSAC Category,Water Exposure,Water Type,Animal Type
0,1971,02,California,Water,Copper,NaN,Confirmed,Restaurant,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
1,1971,06,Arkansas,Water,Hepatitis A,NaN,Confirmed,Store,98,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Other,NaN
2,1971,06,Missouri,Water,Unknown,NaN,Suspected,Subdivision/Neighborhood,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
3,1971,06,Alabama,Water,Selenium,NaN,Confirmed,Unknown,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Individual/Private,NaN
4,1971,06,Vermont,Water,Unknown,NaN,Suspected,Community/municipality,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN


In [10]:
NORS_df['Date'] = NORS_df['Year'].astype('str') + "-" + NORS_df['Month'].astype('str')
column_to_move = NORS_df.pop('Date')
NORS_df.insert(0, 'Date', column_to_move)
NORS_df.drop(columns=['Year', 'Month'], inplace= True)
NORS_df.head()

,Date,State,Primary Mode,Etiology,Serotype or Genotype,Etiology Status,Setting,Illnesses,Hospitalizations,Info On Hospitalizations,Deaths,Info On Deaths,Food Vehicle,Food Contaminated Ingredient,IFSAC Category,Water Exposure,Water Type,Animal Type
0,1971-02,California,Water,Copper,NaN,Confirmed,Restaurant,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
1,1971-06,Arkansas,Water,Hepatitis A,NaN,Confirmed,Store,98,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Other,NaN
2,1971-06,Missouri,Water,Unknown,NaN,Suspected,Subdivision/Neighborhood,2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
3,1971-06,Alabama,Water,Selenium,NaN,Confirmed,Unknown,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Individual/Private,NaN
4,1971-06,Vermont,Water,Unknown,NaN,Suspected,Community/municipality,3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Drinking water,Community,NaN
